In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
from ipywidgets import IntProgress
from IPython.display import display

In [3]:
from dependencies_calculator import DependenciesCalculator
from cost_calculator_factory import CostCalculatorFactory
from multibinary_converter import MultiBinaryConverter
from multidiscrete_converter import MultiDiscreteConverter
from fitness_calculator import FitnessCalculator

In [4]:
from genetic_multibinary_space_config import GeneticMultiBinarySpaceConfig
from genetic_multidiscrete_space_config import GeneticMultiDiscreteSpaceConfig
from genetic_config import GeneticConfig
from genetic_solver import GeneticSolver

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
M = 10 ** 6

In [7]:
read_matrix = lambda path: pd.read_csv(path, index_col=0, encoding='utf-8').to_numpy()

In [8]:
C = read_matrix('../publication_4/script/out/C.csv')

In [9]:
D = read_matrix('../publication_4/script/out/D.csv')

In [10]:
Q = read_matrix('../publication_4/script/out/Q.csv')

In [11]:
R = read_matrix('../publication_4/script/out/R.csv')

In [12]:
constants_file = open('../publication_4/script/out/constants.json', 'r', encoding='utf-8')
constants_data = constants_file.read()
constants_file.close()
constants = json.loads(constants_data)

In [13]:
n = constants['n']
m = constants['m']
l = constants['l']

In [14]:
# solver_name = 'cbc'

In [15]:
calculate_deps_progress = IntProgress(min=0, max=m, value=0, description='deps', bar_style='success')
display(calculate_deps_progress)
deps_calculator = DependenciesCalculator(D, m, calculate_deps_progress)
sum_D = deps_calculator.calculate()

IntProgress(value=0, bar_style='success', description='deps', max=124)

In [16]:
def calculate(k):
    config = GeneticConfig()
    multidiscrete_converter = MultiDiscreteConverter(m, k)
    cost_calculator_factory = CostCalculatorFactory(Q, sum_D, C, R)
    multidiscrete_fitness_calculator = FitnessCalculator(M, cost_calculator_factory, multidiscrete_converter)
    multidiscrete_space_config = GeneticMultiDiscreteSpaceConfig(m, k)
    multidiscrete_config = GeneticConfig()
    multibinary_progress = IntProgress(min=0, max=config.num_generations, value=0, description=str(k), bar_style='success')
    multidiscrete_genetic_solver = GeneticSolver(config, multidiscrete_space_config, multidiscrete_fitness_calculator, multibinary_progress)
    # display(multibinary_progress)
    best_solution, best_fitness, best_idx, duration = multidiscrete_genetic_solver.solve()
    A = multidiscrete_converter.convert(best_solution)
    cost_calculator = cost_calculator_factory.get_cost_calculator(A)
    cost = cost_calculator.calculate()
    return cost

In [17]:
def save_k_costs(k, costs):
    file = open('out_4/k_costs.json', 'r', encoding='utf-8')
    data = file.read()
    file.close()
    d = json.loads(data)
    d[str(k)] = costs
    data = json.dumps(d, indent=4)
    file = open('out_4/k_costs.json', 'w', encoding='utf-8')
    file.write(data)
    file.close()

In [18]:
def make(k):
    iterations = 100
    progress = IntProgress(min=0, max=iterations, value=0, description=str(k), bar_style='success')
    display(progress)
    costs = []
    for _ in range(iterations):
        cost = calculate(k)
        costs.append(cost)
        progress.value += 1
    save_k_costs(k, costs)

In [24]:
for k in range(1, 51):
    make(k)

IntProgress(value=0, bar_style='success', description='46')

IntProgress(value=0, bar_style='success', description='47')

IntProgress(value=0, bar_style='success', description='48')

IntProgress(value=0, bar_style='success', description='49')

IntProgress(value=0, bar_style='success', description='50')

In [27]:
k_cost_file = open('out_4/k_costs.json', 'r', encoding='utf-8')
k_cost_data = k_cost_file.read()
k_cost_file.close()

In [28]:
k_cost = json.loads(k_cost_data)

In [30]:
k_cost_df = pd.DataFrame(k_cost)

In [31]:
k_cost_df.describe()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
count,100.0,100.00,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.0000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1500.0,1498.85,1476.880000,1421.260000,1363.030000,1303.110000,1252.040000,1201.750000,1150.340000,1128.280000,...,634.850000,627.1900,615.350000,614.460000,613.29000,608.080000,601.930000,604.360000,596.200000,592.720000
std,0.0,11.50,43.352975,50.704655,56.675838,62.914708,63.582432,53.199904,57.948291,54.295875,...,26.631179,27.1467,28.087301,30.640773,31.51376,29.324613,27.700511,25.052075,26.736273,24.002138
min,1500.0,1385.00,1370.000000,1288.000000,1224.000000,1150.000000,1053.000000,1088.000000,1014.000000,960.000000,...,568.000000,557.0000,554.000000,547.000000,546.00000,540.000000,551.000000,532.000000,542.000000,544.000000
25%,1500.0,1500.00,1500.000000,1392.750000,1319.500000,1260.250000,1215.750000,1164.750000,1112.750000,1091.000000,...,616.000000,609.0000,596.500000,592.750000,587.75000,584.750000,580.000000,586.000000,579.000000,575.000000
50%,1500.0,1500.00,1500.000000,1413.000000,1379.000000,1304.000000,1256.000000,1208.000000,1152.000000,1141.500000,...,635.000000,622.0000,612.000000,610.000000,615.00000,608.000000,599.500000,603.500000,594.000000,592.000000
75%,1500.0,1500.00,1500.000000,1448.500000,1405.250000,1346.000000,1300.750000,1234.000000,1192.750000,1168.250000,...,653.500000,646.5000,632.000000,631.000000,631.50000,625.250000,623.250000,624.250000,613.250000,608.000000
max,1500.0,1500.00,1500.000000,1500.000000,1500.000000,1427.000000,1359.000000,1340.000000,1257.000000,1229.000000,...,708.000000,721.0000,701.000000,703.000000,678.00000,688.000000,664.000000,653.000000,664.000000,658.000000


In [32]:
k_cost_df.to_csv('out_4/dataframe.csv')